In [256]:
import pandas as pd
import functools
import numpy as np

In [321]:
x

,First Name,Last Name,Display Name,Nickname,E-mail Address,E-mail 2 Address,E-mail 3 Address,Home Phone,Business Phone,Home Fax,...,Notes,Birthday,Anniversary,Gender,Web Page,Web Page 2,Categories,Unnamed: 37,Unnamed: 38,Unnamed: 39
4052,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4053,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4054,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4055,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4056,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4057,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
c = pd.read_csv("contacts.csv")

v = c.loc[~c["First Name"].isnull() & c["First Name"].str.contains("'"),["First Name","Last Name"]]
v = v.values.tolist()
def splitname(name):
    n = name.split(" ")
    if len(n)==1:
        return [n[0],None]
    return [" ".join(n[:-1]), n[-1]]
newNames = [splitname(first.strip("'")) if first[0]=="'" and first[-1]=="'" else [first,last] for first,last in v ]
newNames


[['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ['Allison', 'Bloom'],
 ["St. Mary's Quick Care", 'Adult clinic'],
 ["St. Mary's Quick Care", 'Adult clinic'],
 ['Barnum', None],
 ['Barnum', None],
 ["Lotta's", 'Bakery'],
 ["Lotta's", 'Bakery'],
 ['Camilla', 'Newhagen'],
 ['Camilla', 'Newhagen'],
 ['Claire', 'Pollioni'],
 ['Claire', 'Pollioni'],
 ["Child's delight", nan],
 ["Jonathan's", 'Chinese Restaurant'],
 ["Jonathan's", 'Chinese Restaurant'],
 ['Dante and Mary', 'Menicucci'],
 ['Dante and Mary', 'Menicucci'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ['Davd Koff', '(davidkoff@gmail.com)'],
 ["'Dgteam", nan],
 ["'Dgteam", nan],
 ["Miller's East Coast", 'Deli'],
 ["Miller's East Coast", 'Deli'],
 ['Erin', 'Shallat'],
 ['Erin', 'Shallat'],
 ["Eliza's", nan],
 ["Eliza's", nan],
 ["Giorgio's Pizza", nan],
 ["Giorgio's Pizza", nan],
 ['Michael', None],
 ['Michael

In [386]:
class Contacts:
    def __init__(self):
        self.Contacts = pd.read_csv("contacts.csv")
        self.fixNames()
        b = len(self.Contacts)
        self.Contacts = self.Contacts.drop_duplicates(ignore_index=True)
        dup = b - len(self.Contacts) 
        self.add()
        a = len(self.Contacts)
        print("before:",b, "after:",a, "# of duplicates:",dup, "# compressed:", b-a)
        i = self.Contacts.groupby(["First Name","Last Name"])["Home Phone"].count()
        print("# of different people (unique first and last names):", len(i))
        self.Contacts["Display Name"] = self.Contacts[["First Name","Last Name"]].apply(self.replaceDisplayName,axis=1)
    
    def replaceDisplayName(self,x):
        f = x["First Name"]
        l = x["Last Name"]
        if pd.isnull(f):
            return l
        if pd.isnull(l):
            return f
        return f + " " + l
    
    def splitname(self,name):
        n = name.split(" ")
        if len(n)==1:
            return [n[0],None]
        return [" ".join(n[:-1]), n[-1]]
    def fixNames(self):
        v = self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]].values.tolist()
        newNames = [self.splitname(first.strip("'")) if first[0]=="'" and first[-1]=="'" else [first,last] for first,last in v ]
        self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]] = newNames

    
    def merge(self,person):
        newperson = []
        for feature in person.columns:
            vals = list(person[feature].dropna().unique())
            if len(vals)==0:
                vals = [None]
            newF = functools.reduce(lambda a, b: str(a)+" "+str(b), vals)
            newperson.append(newF)
        return newperson
    def createNewPersonContacts(self, person):
        less_than_one_phone = len(person["Home Phone"].dropna().unique())<=1
        less_than_one_email = len(person["E-mail Address"].dropna().unique())<=1
        if less_than_one_phone and less_than_one_email:
            newperson = [self.merge(person)]
        elif not less_than_one_email and less_than_one_phone:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        elif not less_than_one_phone and less_than_one_email: 
            newperson = person.groupby("Home Phone").apply(self.merge).values
        else:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        newperson = list(newperson)
        return newperson
                
                
    def add(self):
        c = self.Contacts.groupby(["First Name","Last Name"],dropna=False)
        newC = []
        o = 0
        for i,person in c:
            newperson = self.createNewPersonContacts(person) 
            o += len(person)
            newC += newperson
        
        print(len(self.Contacts),o)
            
        self.Contacts = pd.DataFrame(newC,columns=self.Contacts.columns)
        self.Contacts = self.Contacts.sort_values(by=["First Name","Last Name"])

In [387]:
c = Contacts()
c.Contacts.to_csv("newContacts.csv")

4148 4148
before: 6771 after: 3729 # of duplicates: 2623 # compressed: 3042
# of different people (unique first and last names): 1956


In [369]:
c.Contacts

,First Name,Last Name,Display Name,Nickname,E-mail Address,E-mail 2 Address,E-mail 3 Address,Home Phone,Business Phone,Home Fax,...,Notes,Birthday,Anniversary,Gender,Web Page,Web Page 2,Categories,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Allison,Bloom,NaN,NaN,bloomfive@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allison,Bloom,NaN,NaN,bloomfive@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Perry,(Aniko Skyler) Sye,NaN,NaN,perrysye@gmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A Johnson,NaN,NaN,NaN,mmmgmt@gmail.com,NaN,NaN,NaN,NaN,NaN,...,Yuyyyyyyyyyyy is,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Johnson,NaN,NaN,NaN,mmmgmt@gmail.com,NaN,NaN,NaN,NaN,NaN,...,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,Boba Tea,36th And Balboa Near Polo Fields,NaN,NaN,NaN,NaN,NaN,(415) 213-5779,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41541733772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4145,NaN,42 Degree's d,NaN,NaN,NaN,NaN,NaN,4157775558,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4146,Stuart Kesselman Mr Ticket,49er Or Baseball Or Other Tix Tickets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [381]:
np.nan != np.nan

True